In [1]:
import re  

import pdfplumber

pdf = pdfplumber.open("train/atlas.pdf")

In [2]:
import pandas as pd

df = pd.DataFrame(columns=['name', 'page_start', 'page_end', 'Описание', 'Ареал', 'Экология', 'Ресурсы', 'Состов', 'Сырье', 'Фармакология', 'Медицина'])

In [3]:
def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))


In [4]:
def parse_content(page_num):
    page = pdf.pages[page_num]
    names_text = page.crop((0, 0 * float(page.height), 0.45 * float(page.width), 0.95 * float(page.height))).extract_text()
    pages_text = page.crop((0.87 * float(page.width), 0* float(page.height), page.width, 0.95 * float(page.height))).extract_text()

    names_text = names_text.replace('СОД\n', '')

    names_text = names_text.replace('.', '')
    pages_text = pages_text.replace('.', '')
    blocks_names = names_text.split('\n')
    blocks_pages = pages_text.split('\n')
    r = [i for i in range(0, len(blocks_names), 2)]
    p = [i for i in range(1, len(blocks_pages), 2)]
    names = [blocks_names[i] for i in r]
    pages = [int(i)for i in blocks_pages]

    return names, pages

In [5]:
result_names = []
result_pages = []

for i in range(5, 18):
    tn, tp = parse_content(i)
    
    result_names.append(tn)
    result_pages.append(tp)
    

In [6]:
result_names = [item for sublist in result_names for item in sublist]
result_pages = [item for sublist in result_pages for item in sublist]
result_pages.append(617)
result_names.append('not a plant')

In [7]:
for i in range(len(result_names)):
    print(i, result_names[i], result_pages[i])

0 АДЕНОСТИЛЕС РОМБОЛИСТНЫЙ 19
1 АИР ОБЫКНОВЕННЫЙ 21
2 АЛОЭ ДРЕВОВИДНОЕ 24
3 АЛТЕЙ ЛЕКАРСТВЕННЫЙ 27
4 АММИ БОЛЬШАЯ 30
5 АММИ ЗУБНАЯ 33
6 АНИС ОБЫКНОВЕННЫЙ 35
7 АРАЛИЯ ВЫСОКАЯ 37
8 АРНИКА ГОРНАЯ 40
9 АРНИКА ШАМИССО 43
10 АРОНИЯ ЧЕРНОПЛОДНАЯ 45
11 АСТРАГАЛ ШЕРСТИСТОЦВЕТКОВЫЙ 48
12 БАГУЛЬНИК БОЛОТНЫЙ 51
13 БАДАН ТОЛСТОЛИСТНЫЙ 55
14 БАРАНЕЦ ОБЫКНОВЕННЫЙ 59
15 БАРБАРИС ОБЫКНОВЕННЫЙ 61
16 БАРВИНОК МАЛЫЙ 64
17 БАРХАТ АМУРСКИЙ 66
18 БЕДРЕНЕЦ КАМНЕЛОМКА 70
19 БЕЗВРЕМЕННИК ВЕЛИКОЛЕПНЫЙ 73
20 БЕЛЕНА ЧЕРНАЯ 76
21 БЕЛОКОПЫТНИК ГИБРИДНЫЙ 78
22 БЕРЁЗА ПОВИСЛАЯ 80
23 БЕССМЕРТНИК ПЕСЧАНЫЙ 85
24 БОРЕЦ СЕВЕРНЫЙ 89
25 БОЯРЫШНИК КРОВАВО-КРАСНЫЙ 94
26 БРИОФИЛЛУМ ПЕРИСТЫЙ 99
27 БРУСНИКА ОБЫКНОВЕННАЯ 101
28 БУЗИНА ЧЕРНАЯ 105
29 ВАЛЕРИАНА ЛЕКАРСТВЕННАЯ 108
30 ВАСИЛЕК БЛАГОСЛОВЕННЫЙ 112
31 ВАСИЛЕК ПОСЕВНОЙ 114
32 ВАХТА ТРЕХЛИСТНАЯ 116
33 ВЕРБЕНА ЛЕКАРСТВЕННАЯ 118
34 ВЗДУТОПЛОДНИК СИБИРСКИЙ 120
35 ВИНОГРАД КУЛЬТУРНЫЙ 123
36 ВОДОСБОР ОБЫКНОВЕННЫЙ 128
37 ВОЛОДУШКА ЗОЛОТИСТАЯ 130
38 ГАМАМЕЛИС ВИРГИНСКИЙ 133
39 ГАРМА

In [ ]:
errors = []

for i in range(len(result_pages)):
    try:
        st = result_pages[i]
        en = result_pages[i+1]
        contents = ''
        result = {}
        result['name'] = result_names[i]
        result['page_start'] = st
        result['page_end'] = en
        for p in range(st-1, en):

            page = pdf.pages[p]

            y = 0.88*page.height

            if p == (st-1):
                ys = page.rects[0]['pts'][0][1] + 15
                ye = y
            elif (p == (en-1)) and (p != (en-1)):
                ye = page.rects[0]['pts'][2][1]
                ys = 0
            elif p == 94:
                ys = 270
            elif p == 460:
                ye = 630
            else:
                ys = 0
                ye = y
            
            coef = 0.51
            if ((p % 2) == 0):
                coef = 0.49
    
            left = page.crop((0, ys, coef * float(page.width), ye)).extract_text()
            right = page.crop((coef * float(page.width), ys, page.width, ye)).extract_text()

            content = left + ' ' + right

            content = content.replace(' \n', '')
            content = content.replace(' -\n', '')
            content = content.replace(' . ', '. ')
            if (result_names[i] in content) and ('Описание.' in content):
                content = result_names[i] + ' \nОписание.' + content.split('Описание.')[1]
            elif ((result_names[i] in content)): 
                content = ''
            if result_names[i+1] in content:
                content = content.replace(result_names[i+1], '').split('Описание.')[0]
            contents += content



        contents = contents.split(result_names[i+1])[0]
        contents = re.sub(' +', ' ', contents)
        contents = contents.replace('ё', 'е')
        contents = contents.replace('Apeал', 'Ареал')
        contents = contents.split('Описание.')[1].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')

        areal_split = 'Ареал.'

        if i == 24:
            areal_split =  '\nАреал.'
        elif i == 49:
            areal_split =  '\nAреал.'
        
        if i != 57:
            result['Описание'] = contents.split(areal_split)[0]
        
            contents = contents.split(areal_split)[1] 
            result['Ареал'] = contents.split('Экология.')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')

            contents = contents.split('Экология.')[1] 

        else:
            result['Описание'] = contents.split('Экология.')[0]
            result['Ареал'] = ''
            contents = contents.split('Экология.')[1].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')


        if i != 38:
            result['Экология'] = contents.split('Ресурсы.')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')
            contents = contents.split('Ресурсы.')[1] 
            result['Ресурсы'] = contents.split('Химический состав.')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')
            contents = contents.split('Химический состав.')[1]

        else:
            result['Экология'] = contents.split('Химический состав.')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')
            result['Ресурсы'] = ''
            contents = contents.split('Химический состав.')[1]

        chem_split = 'Сырье.'

        if i == 28:
            chem_split =  '\nСырье'

        result['Состов'] = contents.split(chem_split)[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')

        contents = contents.split(chem_split)[1]

        farm_split = 'Фармакологические свойства.'
        if i in [26, 74]:
            farm_split =  'Фармакологическое действие.'

        result['Сырье'] = contents.split(farm_split)[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')

        contents = contents.split(farm_split)[1]
        result['Фармакология'] = contents.split('Применение в медицине.')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')


        contents = contents.split('Применение в медицине.')[1]
        
        if 'Литература\n' in contents:
            result['Медицина'] = contents.split('Литература\n')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')
        else:
            result['Медицина'] = contents.split('Литература')[0].replace('-\n', '').replace('.\n', '. ').replace(',\n', ', ').replace('\n', ' ')
        df = pd.concat([pd.Series(result), df], ignore_index=True, axis=1)
    except:
        errors.append(i)
        

In [ ]:
df.T.to_excel('dataset.xlsx')